In [ ]:
%%capture
! pip install imblearn

In [1]:
import getpass
from predibase import PredibaseClient

token = getpass.getpass("Enter your Predibase API token: ")
pb = PredibaseClient(gateway="http://localhost:8080", token=token)

In [2]:
pb.get_current_user()

User(id=33342128-de60-4612-aff3-fbb369bc5c25, username=joppe, tenant=default)

In [6]:
from ludwig.datasets import creditcard_fraud

df = creditcard_fraud.load()
df.drop(columns=["split"], inplace=True)

In [7]:
dataset = pb.create_dataset_from_df(df, "creditcard_fraud")
# dataset = pb.get_dataset("creditcard_fraud")

Output()

✅ Create Dataset from Dataframe

In [8]:
repo = pb.create_model_repo("creditcard_fraud", exists_ok=True)
draft = repo.create_draft(dataset, "Class")
draft.config['preprocessing'] = {
    'split': {
        'type': 'stratify',
        'column': 'Class',
    },
}

Model repository creditcard_fraud already exists. exists_ok=True, so ignoring.

/usr/local/lib/python3.8/site-packages/dataclasses_json/core.py:171: RuntimeWarning: `NoneType` object value of non-optional type _description detected when decoding ModelRepo.
  warnings.warn(f"`NoneType` object {warning}.", RuntimeWarning)


In [14]:
# without SMOTE
draft.config['preprocessing']['smote'] = False
model = draft.train()

Output()

✅ Train Model

In [20]:
model.metrics.test.Class.roc_auc

0.5958479046821594

In [36]:
# with SMOTE
model = repo.latest()
draft = model.to_draft()
draft.config['preprocessing']['smote'] = True
model_smote = draft.train()

Output()

💥 Train Model

RuntimeError: Failed to train model with status FAILED

Remote error:

activity error (type: PreprocessModel, scheduledEventID: 71, startedEventID: 72, identity: 1436@02cbc78c6a7a): 
Traceback (most recent call last):
  File "/workspaces/predibase/engine/predibase_engine/workflow/util.py", line 53, in wrapped_fn
    return task.result()
  File "/usr/local/lib/python3.8/asyncio/tasks.py", line 695, in _wrap_awaitable
    return (yield from awaitable.__await__())
ray.exceptions.RayTaskError(UnboundLocalError): [36mray::remote_fn()[39m (pid=2441, ip=172.1.0.4)
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 1666, in preprocess_for_training
    processed = data_format_processor.preprocess_for_training(
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 236, in preprocess_for_training
    return _preprocess_df_for_training(
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 1909, in _preprocess_df_for_training
    if preprocessing_params["smote"]:
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 1439, in smote
    X_train_oversampled, y_train_oversampled = sm.fit_resample(dataset_df.drop(columns=), dataset_df)
  File "/home/vscode/.local/lib/python3.8/site-packages/imblearn/base.py", line 83, in fit_resample
    output = self._fit_resample(X, y)
  File "/home/vscode/.local/lib/python3.8/site-packages/imblearn/over_sampling/_smote/base.py", line 505, in 
_fit_resample
    X_categorical = X[:, self.categorical_features_]
IndexError: arrays used as indices must be of integer (or boolean) type

During handling of the above exception, another exception occurred:

[36mray::remote_fn()[39m (pid=2441, ip=172.1.0.4)
  File "/workspaces/predibase/engine/predibase_engine/workflow/util.py", line 38, in remote_fn
    return fn(ctx, *args, **kwargs)
  File "/workspaces/predibase/engine/predibase_engine/workflow/activities.py", line 59, in preprocess_model
    success = preprocess_step(
  File "/workspaces/predibase/engine/predibase_engine/train/__init__.py", line 129, in preprocess_step
    model.preprocess(
  File "/workspaces/ludwig/ludwig/api.py", line 1408, in preprocess
    callback.on_preprocess_end(proc_training_set, proc_validation_set, proc_test_set, training_set_metadata)
UnboundLocalError: local variable 'proc_training_set' referenced before assignment
: [36mray::remote_fn()[39m (pid=2441, ip=172.1.0.4)
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 1666, in preprocess_for_training
    processed = data_format_processor.preprocess_for_training(
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 236, in preprocess_for_training
    return _preprocess_df_for_training(
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 1909, in _preprocess_df_for_training
    if preprocessing_params["smote"]:
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 1439, in smote
    X_train_oversampled, y_train_oversampled = sm.fit_resample(dataset_df.drop(columns=), dataset_df)
  File "/home/vscode/.local/lib/python3.8/site-packages/imblearn/base.py", line 83, in fit_resample
    output = self._fit_resample(X, y)
  File "/home/vscode/.local/lib/python3.8/site-packages/imblearn/over_sampling/_smote/base.py", line 505, in 
_fit_resample
    X_categorical = X[:, self.categorical_features_]
IndexError: arrays used as indices must be of integer (or boolean) type

During handling of the above exception, another exception occurred:

[36mray::remote_fn()[39m (pid=2441, ip=172.1.0.4)
  File "/workspaces/predibase/engine/predibase_engine/workflow/util.py", line 38, in remote_fn
    return fn(ctx, *args, **kwargs)
  File "/workspaces/predibase/engine/predibase_engine/workflow/activities.py", line 59, in preprocess_model
    success = preprocess_step(
  File "/workspaces/predibase/engine/predibase_engine/train/__init__.py", line 129, in preprocess_step
    model.preprocess(
  File "/workspaces/ludwig/ludwig/api.py", line 1408, in preprocess
    callback.on_prep

RuntimeError: Failed to train model with status FAILED

Remote error:

activity error (type: PreprocessModel, scheduledEventID: 71, startedEventID: 72, identity: 1436@02cbc78c6a7a): Traceback (most recent call last):
  File "/workspaces/predibase/engine/predibase_engine/workflow/util.py", line 53, in wrapped_fn
    return task.result()
  File "/usr/local/lib/python3.8/asyncio/tasks.py", line 695, in _wrap_awaitable
    return (yield from awaitable.__await__())
ray.exceptions.RayTaskError(UnboundLocalError): [36mray::remote_fn()[39m (pid=2441, ip=172.1.0.4)
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 1666, in preprocess_for_training
    processed = data_format_processor.preprocess_for_training(
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 236, in preprocess_for_training
    return _preprocess_df_for_training(
  File "/workspaces/ludwig/ludwig/data/preprocessing.py", line 1909, in _preprocess_df_for_training
    if preprocessing_params["smote"]:
  F